Dropout的目的也是为了衰减权重，只不过dropout方法不是在输入层进行衰减，而是相当于将中间的隐藏层分成了几个不同深度的子层，这样的训练效果可能更好。

实现方法是设置一个概率p来进行概率丢弃。

In [1]:
import torch
import d2l as d2l
from torch import nn

In [4]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    if dropout == 1:
        return torch.zeros_like(X)
    if dropout == 0:
        return X
    mask = (torch.rand(X.shape) > dropout).float()  # 概率为dropout时返回0
    return mask * X / (1.0 - dropout)

In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((2, 8))
print(X)
print(dropout_layer(X, 0.))
print(dropout_layer(X, 0.5))
print(dropout_layer(X, 1.))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  0.,  4.,  6.,  8.,  0.,  0., 14.],
        [ 0., 18., 20., 22.,  0., 26.,  0., 30.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [6]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

In [9]:
# 定义模型
dropout1 = 0.5
dropout2 = 0.2


class Net(nn.Module):
    def __init__(self,
                 num_inputs,
                 num_outputs,
                 num_hiddens1,
                 num_hiddens2,
                 is_training=True):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.is_training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1, num_inputs))))
        if self.is_training == True:
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.is_training == True:
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2)
        return out


net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)

In [10]:
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

epoch： 1, loss： 0.8922, train_acc： 0.672, test_acc： 0.765
epoch： 2, loss： 0.5247, train_acc： 0.808, test_acc： 0.740
epoch： 3, loss： 0.5154, train_acc： 0.816, test_acc： 0.757
epoch： 4, loss： 0.4383, train_acc： 0.839, test_acc： 0.819
epoch： 5, loss： 0.4155, train_acc： 0.848, test_acc： 0.834
epoch： 6, loss： 0.3913, train_acc： 0.857, test_acc： 0.831
epoch： 7, loss： 0.3800, train_acc： 0.861, test_acc： 0.803
epoch： 8, loss： 0.3667, train_acc： 0.865, test_acc： 0.853
epoch： 9, loss： 0.3526, train_acc： 0.872, test_acc： 0.858
epoch： 10, loss： 0.3443, train_acc： 0.873, test_acc： 0.848


In [14]:
# 简洁实现
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 256), nn.ReLU(),
                    nn.Dropout(dropout1), nn.Linear(256, 256), nn.ReLU(),
                    nn.Dropout(dropout2), nn.Linear(256, 10))


def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)


net.apply(init_weights)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

epoch： 1, loss： 1.1316, train_acc： 0.560, test_acc： 0.736
epoch： 2, loss： 0.5774, train_acc： 0.787, test_acc： 0.773
epoch： 3, loss： 0.4820, train_acc： 0.825, test_acc： 0.733
epoch： 4, loss： 0.4473, train_acc： 0.835, test_acc： 0.743
epoch： 5, loss： 0.4167, train_acc： 0.848, test_acc： 0.842
epoch： 6, loss： 0.3911, train_acc： 0.857, test_acc： 0.813
epoch： 7, loss： 0.3763, train_acc： 0.861, test_acc： 0.853
epoch： 8, loss： 0.3643, train_acc： 0.865, test_acc： 0.866
epoch： 9, loss： 0.3521, train_acc： 0.869, test_acc： 0.850
epoch： 10, loss： 0.3417, train_acc： 0.873, test_acc： 0.834
